In [4]:
import keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "D:\Programs and Documents\Anaconda3 4.4.0\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module(mname)
  File "D:\Programs and Documents\Anaconda3 4.4.0\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 978, in _gcd_import
  File "<frozen importlib._bootstrap>", line 961, in _find_and_load
  File "<frozen importlib._bootstrap>", line 950, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 648, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 560, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
ImportError: DLL load failed: The specified module could not be found.

During handling o

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.wrappers.scikit_learn import KerasRegressor
%matplotlib inline

Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

In [29]:
# read training data 
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [30]:
# read test data
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [31]:
# read ground truth data
truth_df = pd.read_csv('PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [32]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [33]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [23]:
#MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                           columns=cols_normalize, 
                           index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

KeyError: 'cycle'

In [9]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                           columns=cols_normalize, 
                           index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,36.832409,625.500000,-100.0,-518.67,-192.971331,-34.243064,-23.328504,-14.62,...,-3511.263841,-41.798988,-30.842382,-0.03,-32.305556,-2388.0,-100.0,-29.133225,-30.699208,0.00000
1,1,2,20.317677,208.833333,-100.0,-518.67,-193.090180,-34.241562,-23.329299,-14.62,...,-3511.198962,-41.798613,-31.211008,-0.03,-32.298611,-2388.0,-100.0,-29.037077,-30.664696,0.00277
2,1,3,30.226516,486.611111,-100.0,-518.67,-193.022137,-34.242280,-23.327612,-14.62,...,-3511.263841,-41.798867,-30.266495,-0.03,-32.298611,-2388.0,-100.0,-29.001022,-30.617029,0.00554
3,1,4,43.108006,417.166667,-100.0,-518.67,-193.023951,-34.243620,-23.326165,-14.62,...,-3511.220588,-41.798792,-31.042240,-0.03,-32.312500,-2388.0,-100.0,-29.049096,-30.698826,0.00831
4,1,5,33.859757,417.166667,-100.0,-518.67,-193.017601,-34.242161,-23.327447,-14.62,...,-3511.263841,-41.798881,-30.873471,-0.03,-32.319444,-2388.0,-100.0,-29.055105,-30.623893,0.01108


In [10]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [11]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()
#t_plot = test_df[test_df['id'] == 1]
#plt.plot(t_plot['RUL'])
#test_df.head()
#t_plot = test_df[test_df['id'] == 2]
#plt.plot(t_plot['RUL'])
#test_df.head()
#t_plot = test_df[test_df['id'] == 3]
#plt.plot(t_plot['RUL'])

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,36.832409,625.500000,-100.0,-518.67,-192.971331,-34.243064,-23.328504,-14.62,...,-41.798988,-30.842382,-0.03,-32.305556,-2388.0,-100.0,-29.133225,-30.699208,0.00000,142
1,1,2,20.317677,208.833333,-100.0,-518.67,-193.090180,-34.241562,-23.329299,-14.62,...,-41.798613,-31.211008,-0.03,-32.298611,-2388.0,-100.0,-29.037077,-30.664696,0.00277,141
2,1,3,30.226516,486.611111,-100.0,-518.67,-193.022137,-34.242280,-23.327612,-14.62,...,-41.798867,-30.266495,-0.03,-32.298611,-2388.0,-100.0,-29.001022,-30.617029,0.00554,140
3,1,4,43.108006,417.166667,-100.0,-518.67,-193.023951,-34.243620,-23.326165,-14.62,...,-41.798792,-31.042240,-0.03,-32.312500,-2388.0,-100.0,-29.049096,-30.698826,0.00831,139
4,1,5,33.859757,417.166667,-100.0,-518.67,-193.017601,-34.242161,-23.327447,-14.62,...,-41.798881,-30.873471,-0.03,-32.319444,-2388.0,-100.0,-29.055105,-30.623893,0.01108,138


In [12]:
# pick a large window size of 50 cycles
# sequence_length = 50

In [13]:
# preparing data for visualizations 
# window of 50 cycles prior to a failure point for engine id 3
# engine_id3 = test_df[test_df['id'] == 3]
# engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
# cols1 = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
# engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
# cols2 = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
# engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

In [14]:
# plotting sensor data for engine ID 3 prior to a failure point - sensors 1-10 
# ax1 = engine_id3_50cycleWindow1.plot(subplots=True, sharex=True, figsize=(20,20))

In [15]:
# plotting sensor data for engine ID 3 prior to a failure point - sensors 11-21 
# ax2 = engine_id3_50cycleWindow2.plot(subplots=True, sharex=True, figsize=(20,20))

In [16]:
# function to reshape features into (samples, time steps, features) 
# def gen_sequence(id_df, seq_length, seq_cols):
#     """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
#     we need to drop those which are below the window-length. An alternative would be to pad sequences so that
#     we can use shorter ones """
#     data_array = id_df[seq_cols].values
#     num_elements = data_array.shape[0]
#     for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
#         yield data_array[start:stop, :]

In [17]:
# pick the feature columns 
# sensor_cols = ['s' + str(i) for i in range(1,22)]
# sequence_cols = ['setting1', 'setting2', 'setting3']
# sequence_cols.extend(sensor_cols)

In [18]:
# generator for the sequences
# seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
#            for id in train_df['id'].unique())

In [19]:
# generate sequences and convert to numpy array
# seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
# seq_array.shape

In [20]:
# function to generate labels
# def gen_labels(id_df, seq_length, label):
#     data_array = id_df[label].values
#     num_elements = data_array.shape[0]
#     return data_array[seq_length:num_elements, :]

In [22]:
# generate labels
# label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
#              for id in train_df['id'].unique()]
# label_array = np.concatenate(label_gen).astype(np.float32)
label_array = train_df['RUL']
label_array.shape
train_df.drop('RUL', axis=1, inplace=True)
train_df.drop('id', axis=1, inplace=True)
train_df.drop('cycle', axis=1, inplace=True)

KeyError: 'RUL'

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(solver='adam', max_iter=100, activation='tanh',
                           random_state=1, learning_rate_init=0.01,
                           batch_size=200)

# catch convergence warning
mlp.fit(train_df, label_array)
pred1 = mlp.predict(train_df)
print(pred1)
#     for i in range(100):
#         mlp.partial_fit(X, y)

#     pred2 = mlp.predict(X)
#     assert_almost_equal(pred1, pred2, decimal=2)
#     score = mlp.score(X, y)


[ 201.34160218  200.13388104  207.25562748 ...,    7.79027859    7.51929363
    7.50050261]


In [12]:
# build the network
nb_features = train_df.shape[1]
nb_out = label_array.shape[0]

model = Sequential()

model.add(Dense(nb_features, 
         input_dim=nb_features,
         kernel_initializer='normal',
         activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2000,
          activation='relu',      
          kernel_initializer='normal'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [13]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                650       
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2000)              52000     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2001      
Total params: 54,651
Trainable params: 54,651
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
%%time
# fit the network
model.fit(train_df.as_matrix(), label_array, epochs=100, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = None)

Train on 19599 samples, validate on 1032 samples
Epoch 1/100
19599/19599 [==============================] - 4s - loss: 0.0197 - mean_absolute_error: 0.1038 - val_loss: 0.0158 - val_mean_absolute_error: 0.0931
Epoch 2/100
19599/19599 [==============================] - 1s - loss: 0.0122 - mean_absolute_error: 0.0810 - val_loss: 0.0161 - val_mean_absolute_error: 0.0893
Epoch 3/100
19599/19599 [==============================] - 1s - loss: 0.0114 - mean_absolute_error: 0.0775 - val_loss: 0.0152 - val_mean_absolute_error: 0.0924
Epoch 4/100
19599/19599 [==============================] - 0s - loss: 0.0111 - mean_absolute_error: 0.0760 - val_loss: 0.0151 - val_mean_absolute_error: 0.0867
Epoch 5/100
19599/19599 [==============================] - 0s - loss: 0.0110 - mean_absolute_error: 0.0760 - val_loss: 0.0150 - val_mean_absolute_error: 0.0866
Epoch 6/100
19599/19599 [==============================] - 1s - loss: 0.0108 - mean_absolute_error: 0.0751 - val_loss: 0.0147 - val_mean_absolute_error

In [15]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(train_df.as_matrix())
y_true = label_array
# print(y_true)
print(y_pred[1:10, 0])

20480/20631 [============================>.] - ETA: 0s[1 1 1 1 1 1 1 1 1]


In [16]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

ValueError: Can't handle mix of continuous and binary

In [ ]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

In [ ]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]


In [ ]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape
print(len(label_array_test_last))
file1 = open("label.csv","a")  
file1.write(str(label_array_test_last))
file1.write(" ")
file1.close()
print("created label.csv")

In [ ]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

In [ ]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

In [ ]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

In [ ]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

In [ ]:
results_df = pd.DataFrame([[scores_test[1],precision_test,recall_test,f1_test],
                          [0.94, 0.952381, 0.8, 0.869565]],
                         columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'],
                         index = ['LSTM-final',
                                 'LSTM-full'])
results_df